## <center>Projeto de Previsão de Preços para Casas</center>

**Nota importante**: Essa demonstração irá pular alguns passos demonstrados no notebook anterior (01-modelo-clasificacao) e focar apenas nos aspectos mais importantes de um projeto de modelagem preditiva de regressão.

Começamos com a importação dos pacotes necessários.

In [75]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# importamos apenas os métodos necessarios do pacote sklearn, utilizados para o processo de machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


# estilização dos plots
plt.style.use('ggplot')

Vamos carregar os datasets que serão utilizados nessa demonstração. Os mesmos foram disponibilizados junto do notebook.

In [76]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

Visualização dos datasets:

In [77]:
df_train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [78]:
df_test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [79]:
print(df_train.shape)
print(df_test.shape)


(1460, 81)
(1459, 80)


In [80]:
df_train.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
                  ...   
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual       object
TotRmsAbvGrd       int64
Functional        object
Fireplaces         int64
FireplaceQu       object
GarageType        object
GarageYrBlt      float64


In [81]:
df_test.dtypes

Id                 int64
MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
Alley             object
LotShape          object
LandContour       object
Utilities         object
LotConfig         object
LandSlope         object
Neighborhood      object
Condition1        object
Condition2        object
BldgType          object
HouseStyle        object
OverallQual        int64
OverallCond        int64
YearBuilt          int64
YearRemodAdd       int64
RoofStyle         object
RoofMatl          object
Exterior1st       object
Exterior2nd       object
MasVnrType        object
MasVnrArea       float64
ExterQual         object
ExterCond         object
Foundation        object
                  ...   
HalfBath           int64
BedroomAbvGr       int64
KitchenAbvGr       int64
KitchenQual       object
TotRmsAbvGrd       int64
Functional        object
Fireplaces         int64
FireplaceQu       object
GarageType        object


In [82]:
df_train.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,730.500000,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,421.610009,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,1.000000,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,365.750000,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,730.500000,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,1095.250000,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,1460.000000,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


Notas:
* Dados são de casas vendidas entre 2006 e 2010
* Valor mais baixo: 34900, mais alto: 755000


Avaliação da correlação das colunas com a coluna de preço.

In [83]:
df_train.corr()['SalePrice'].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.790982
GrLivArea        0.708624
GarageCars       0.640409
GarageArea       0.623431
TotalBsmtSF      0.613581
1stFlrSF         0.605852
FullBath         0.560664
TotRmsAbvGrd     0.533723
YearBuilt        0.522897
YearRemodAdd     0.507101
GarageYrBlt      0.486362
MasVnrArea       0.477493
Fireplaces       0.466929
BsmtFinSF1       0.386420
LotFrontage      0.351799
WoodDeckSF       0.324413
2ndFlrSF         0.319334
OpenPorchSF      0.315856
HalfBath         0.284108
LotArea          0.263843
BsmtFullBath     0.227122
BsmtUnfSF        0.214479
BedroomAbvGr     0.168213
ScreenPorch      0.111447
PoolArea         0.092404
MoSold           0.046432
3SsnPorch        0.044584
BsmtFinSF2      -0.011378
BsmtHalfBath    -0.016844
MiscVal         -0.021190
Id              -0.021917
LowQualFinSF    -0.025606
YrSold          -0.028923
OverallCond     -0.077856
MSSubClass      -0.084284
EnclosedPorch   -0.128578
KitchenAbvGr    -0.135907
Name: SalePr

Notas:
* Correlação alta: *OverallQual, GrLivArea*
* Correlação moderada relevante: *GarageCars, GarageArea, TotalBsmtSF, 1stFlrSF*
* Correlação moderada fraca: *FullBath, TotRmsAbvGrd, YearBuilt, YearRemodAdd*

Vamos filtrar os datasets para utilizar apenas as colunas numéricas.

In [84]:
df_train = df_train.select_dtypes(include=np.number)
df_test = df_test.select_dtypes(include=np.number)

O próximo passo é tratar os valores nulos,

In [85]:
df_train.isnull().sum().sort_values(ascending=False)

LotFrontage      259
GarageYrBlt       81
MasVnrArea         8
BsmtFinSF1         0
LowQualFinSF       0
2ndFlrSF           0
1stFlrSF           0
TotalBsmtSF        0
BsmtUnfSF          0
BsmtFinSF2         0
SalePrice          0
BsmtFullBath       0
YearRemodAdd       0
YearBuilt          0
OverallCond        0
OverallQual        0
LotArea            0
MSSubClass         0
GrLivArea          0
BsmtHalfBath       0
YrSold             0
FullBath           0
MoSold             0
MiscVal            0
PoolArea           0
ScreenPorch        0
3SsnPorch          0
EnclosedPorch      0
OpenPorchSF        0
WoodDeckSF         0
GarageArea         0
GarageCars         0
Fireplaces         0
TotRmsAbvGrd       0
KitchenAbvGr       0
BedroomAbvGr       0
HalfBath           0
Id                 0
dtype: int64

Verificando o dicionário de dados, conferimos que o campo GarageYrBlt refere-se ao ano de construção de garagem. Porém algumas casas não tem garagem, vamos avaliar se o numero alto de nulos é devido a isso.

In [86]:
# Calcula o numero de valores nulos para linhas com GarageArea nula
df_check = pd.read_csv('train.csv')
print(len(df_check[(df_check['GarageType'].notnull()) & (df_check['GarageYrBlt'].isnull())]['GarageYrBlt']))
del df_check

0


Observamos que o caso de nulos para GarageYrBlt é para casas sem garagem.

Vamos então interpolar os valores desses campos, para que sejam preenchidos com base nas linhas anteriores e proximas a eles.

In [87]:
df_train = df_train.interpolate()
df_test = df_test.interpolate()

df_train.isnull().sum()

Id               0
MSSubClass       0
LotFrontage      0
LotArea          0
OverallQual      0
OverallCond      0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtFinSF2       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
LowQualFinSF     0
GrLivArea        0
BsmtFullBath     0
BsmtHalfBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
TotRmsAbvGrd     0
Fireplaces       0
GarageYrBlt      0
GarageCars       0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
3SsnPorch        0
ScreenPorch      0
PoolArea         0
MiscVal          0
MoSold           0
YrSold           0
SalePrice        0
dtype: int64

Vamos efetur a normalização dos valores de venda.

In [88]:
df_train['SalePrice'] = np.log(df_train['SalePrice'])
df_train.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1,60,65.0,8450,7,5,2003,2003,196.0,706,...,0,61,0,0,0,0,0,2,2008,12.247694
1,2,20,80.0,9600,6,8,1976,1976,0.0,978,...,298,0,0,0,0,0,0,5,2007,12.109011
2,3,60,68.0,11250,7,5,2001,2002,162.0,486,...,0,42,0,0,0,0,0,9,2008,12.317167
3,4,70,60.0,9550,7,5,1915,1970,0.0,216,...,0,35,272,0,0,0,0,2,2006,11.849398
4,5,60,84.0,14260,8,5,2000,2000,350.0,655,...,192,84,0,0,0,0,0,12,2008,12.429216


Vamos remover a coluna ID, pois não é necessária.

In [89]:
df_train = df_train.drop('Id', axis=1)

Vamos agora criar os datasets de treino e teste

In [90]:
X = df_train.drop('SalePrice', axis=1)
y = df_train['SalePrice']

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=.33)

Em seguida, efetuamos o treinamento.

In [92]:
reg = LinearRegression() # inicia instancia de regressor linear
reg.fit(X_train, y_train) # método fit efetua treinamento do modelo com o dataset fornecido

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [93]:
# print do score do treinamento
print('Score de treinamento: {:.2%}'.format(reg.score(X_train, y_train)))

# print do score com dataset de teste
print('Score de treinamento: {:.2%}'.format(reg.score(X_test, y_test)))

Score de treinamento: 86.37%
Score de treinamento: 86.28%


Vamos experimentar a eficiencia do modelo na base df_test

In [95]:
# Criando a coluna de preços previstos a partir do modelo
# np.exp de-normaliza o valor de venda com uma função matemática de exponenciação.
df_test['SalePrice'] = np.exp(reg.predict(df_test.drop('Id', axis=1)))

df_test.head()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,1461,20,80.0,11622,5,6,1961,1961,0.0,468.0,...,140,0,0,0,120,0,0,6,2010,119157.135943
1,1462,20,81.0,14267,6,6,1958,1958,108.0,923.0,...,393,36,0,0,0,0,12500,6,2010,120851.670445
2,1463,60,74.0,13830,5,5,1997,1998,0.0,791.0,...,212,34,0,0,0,0,0,3,2010,168218.023445
3,1464,60,78.0,9978,6,6,1998,1998,20.0,602.0,...,360,36,0,0,0,0,0,6,2010,195638.188570
4,1465,120,43.0,5005,8,5,1992,1992,0.0,263.0,...,0,82,0,0,144,0,0,1,2010,181909.367798


Vamos visualizar algumas informa;'oes sobre o df_test

In [96]:
df_test.describe()

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,...,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000,1459.000000
mean,2190.000000,57.378341,68.881768,9819.161069,6.078821,5.553804,1971.357779,1983.662783,100.933173,438.953393,...,93.174777,48.313914,24.243317,1.794380,17.064428,1.744345,58.167923,6.104181,2007.769705,176861.067145
std,421.321334,42.746880,21.713993,4955.517327,1.436812,1.113740,30.390071,21.130467,177.172465,455.212307,...,127.744882,68.883364,67.227765,20.207842,56.609763,30.491646,630.806978,2.722432,1.301740,71512.371229
min,1461.000000,20.000000,21.000000,1470.000000,1.000000,1.000000,1879.000000,1950.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,57922.541401
25%,1825.500000,20.000000,59.000000,7391.000000,5.000000,5.000000,1953.000000,1963.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,2007.000000,126443.052473
50%,2190.000000,50.000000,68.000000,9399.000000,6.000000,5.000000,1973.000000,1992.000000,0.000000,350.000000,...,0.000000,28.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,159262.792665
75%,2554.500000,70.000000,80.000000,11517.500000,7.000000,6.000000,2001.000000,2004.000000,164.000000,752.000000,...,168.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,209073.055960
max,2919.000000,190.000000,200.000000,56600.000000,10.000000,9.000000,2010.000000,2010.000000,1290.000000,4010.000000,...,1424.000000,742.000000,1012.000000,360.000000,576.000000,800.000000,17000.000000,12.000000,2010.000000,749810.351243


In [102]:
df_test.corr()['SalePrice'].sort_values(ascending=False)

SalePrice        1.000000
OverallQual      0.844821
GrLivArea        0.750511
GarageCars       0.729287
GarageArea       0.708924
TotalBsmtSF      0.656559
1stFlrSF         0.649382
YearBuilt        0.631370
YearRemodAdd     0.593134
GarageYrBlt      0.589451
FullBath         0.579204
MasVnrArea       0.533969
Fireplaces       0.522618
TotRmsAbvGrd     0.517982
BsmtFinSF1       0.488949
WoodDeckSF       0.382468
HalfBath         0.345456
BsmtFullBath     0.343941
LotArea          0.326448
LotFrontage      0.322740
OpenPorchSF      0.316392
2ndFlrSF         0.255165
BedroomAbvGr     0.159056
BsmtUnfSF        0.146961
ScreenPorch      0.122231
MiscVal          0.119918
MoSold           0.056874
3SsnPorch        0.036760
BsmtFinSF2       0.022422
PoolArea        -0.007820
YrSold          -0.021723
LowQualFinSF    -0.033248
BsmtHalfBath    -0.045858
Id              -0.047522
KitchenAbvGr    -0.078877
MSSubClass      -0.098980
OverallCond     -0.119239
EnclosedPorch   -0.148327
Name: SalePr

Por fim, basta salvar o modelo para utilização futura.  

In [105]:
import pickle

pickle.dump(reg, open('modelo.sav', 'wb'))

Esse dataset é fornecido pelo [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques#evaluation).  

Essa metodologia possui uma pontuação de 0.15143 ([Root-Mean-Squared-Error (RMSE)](https://en.wikipedia.org/wiki/Root-mean-square_deviation)) no competição, o que não é muito boa, pois é uma método simples apenas para demonstrar o processo de criação de um modelo de machine learning. Porém, a partir daqui, você já pode experimentar novas técnicas e alternativas utilizando seu conhecimento de data cleaning, data wrangling e feature engineering. 